In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

import glob
import pandas as pd

Analysis of datasets

In [ ]:
modis_sst_revlat = xr.open_dataset('dati/mist/old/dincae/modis_sst_revlat.nc')
modis_sst_revlat

In [ ]:
modis_sst_revlat_add_clouds = xr.open_dataset('dati/mist/old/dincae/modis_sst_revlat_add_clouds.nc')
modis_sst_revlat_add_clouds

In [ ]:
msr_sst = modis_sst_revlat['sst']
print('shape of sst data:', msr_sst.shape)

msr_qual = modis_sst_revlat['qual']
print('shape of qual data:', msr_qual.shape)

msr_sst_t = modis_sst_revlat['sst_t']
print('shape of sst_t data:', msr_sst_t.shape)

msr_mask = modis_sst_revlat['mask']
print('shape of sst mask:', msr_mask.shape)

msr_count_nomissing = modis_sst_revlat['count_nomissing']
print('shape of count_nomissing data:', msr_count_nomissing.shape)

In [ ]:
msrc_sst = modis_sst_revlat_add_clouds['sst']
print('shape of sst data:', msrc_sst.shape)

msrc_qual = modis_sst_revlat_add_clouds['qual']
print('shape of qual data:', msrc_qual.shape)

msrc_sst_t = modis_sst_revlat_add_clouds['sst_t']
print('shape of sst_t data:', msrc_sst_t.shape)

msrc_mask = modis_sst_revlat_add_clouds['mask']
print('shape of sst mask:', msrc_mask.shape)

msrc_time_index_validation = modis_sst_revlat_add_clouds['time_index_validation']
print('shape of time_index_validation data:', msrc_time_index_validation.shape)

In [ ]:
# Count quality levels over both datasets

# msr_qual = msr_qual.values
# msrc_qual = msrc_qual.values
# print(msr_qual.shape)
# print(msrc_qual.shape)

def analyze_quality(qual_array):
    # Flatten the array to make counting easier
    flattened = qual_array.flatten()
    
    # Count the total number of valid (non-NaN) datapoints
    total_valid = np.count_nonzero(~np.isnan(flattened))
    
    # Initialize a dictionary to hold counts and percentiles for each quality level
    quality_counts = {i: np.count_nonzero(flattened == i) for i in range(6)}
    quality_percentiles = {i: (quality_counts[i] / total_valid) * 100 for i in range(6)}
    
    return quality_counts, quality_percentiles

# Analyze quality for msr and msrc
msr_quality_counts, msr_quality_percentiles = analyze_quality(msr_qual)
msrc_quality_counts, msrc_quality_percentiles = analyze_quality(msrc_qual)

# Print the results

# Print the results
print("Quality counts for MSR:", msr_quality_counts)
print("Quality percentiles for MSR:", msr_quality_percentiles)
print("Quality counts for MSRC:", msrc_quality_counts)
print("Quality percentiles for MSRC:", msrc_quality_percentiles)

Plotting

In [ ]:
indexToPlot = 3235
# index1: 3235
# index2: 3236

In [ ]:
msr_sst_t[indexToPlot].plot()
#msrc_sst_t[indexToPlot].plot()

In [ ]:
#plot msr dataset
plt.figure(figsize=(16, 8))

plt.subplot(2, 3, 1)
plt.title('sst')
plt.imshow(msr_sst[indexToPlot, :, :], cmap='viridis', origin='lower')
plt.colorbar()

plt.subplot(2, 3, 2)
plt.title('qual')
plt.imshow(msr_qual[indexToPlot, :, :], cmap='tab20b', origin='lower')
plt.colorbar()

plt.subplot(2, 3, 3)
plt.title('sst_t')
plt.imshow(msr_sst_t[indexToPlot, :, :], cmap='viridis', origin='lower')
plt.colorbar()

plt.subplot(2, 3, 4)
plt.title('mask')
plt.imshow(msr_mask, cmap='viridis', origin='lower')
plt.colorbar()

plt.subplot(2, 3, 5)
plt.title('count_nomissing')
plt.imshow(msr_count_nomissing, cmap='YlOrBr', origin='lower')
plt.colorbar()

plt.show()

In [ ]:
#plot msrc dataset
plt.figure(figsize=(10, 8))

plt.subplot(2, 2, 1)
plt.title('sst')
plt.imshow(msrc_sst[indexToPlot, :, :], cmap='viridis', origin='lower')
plt.colorbar()

plt.subplot(2, 2, 2)
plt.title('qual')
plt.imshow(msrc_qual[indexToPlot, :, :], cmap='tab20b', origin='lower')
plt.colorbar()

plt.subplot(2, 2, 3)
plt.title('sst_t')
plt.imshow(msrc_sst_t[indexToPlot, :, :], cmap='viridis', origin='lower')
plt.colorbar()

plt.subplot(2, 2, 4)
plt.title('mask')
plt.imshow(msrc_mask, cmap='viridis', origin='lower')
plt.colorbar()

# plt.subplot(2, 3, 5)
# plt.title('time_index_validation')
# msrc_time_index_validation.plot()

plt.show()

In [ ]:
#properly plot time_index_validation

plt.figure(figsize=(16, 8))
plt.title('time_index_validation')
msrc_time_index_validation.plot()
plt.show()

# sorted_time_index_validation = np.sort(msrc_time_index_validation.values)
# print('time_index_validation:', sorted_time_index_validation)

In [ ]:
# Check if the masks and sst in the two datasets (msr and msrc) are the same

for index in range(msr_sst.shape[0]):
    mask1 = np.isnan(msr_sst[index, :, :])
    mask2 = np.isnan(msrc_sst[index, :, :])

    temp = (mask1 == mask2)
    if not np.all(temp):
        plt.figure(figsize=(10, 8))
        plt.subplot(1, 3, 1)
        plt.title('sst of msr')
        plt.imshow(mask1, cmap='viridis', origin='lower')
        plt.subplot(1, 3, 2)
        plt.title('sst of msrc')
        plt.imshow(mask2, cmap='viridis', origin='lower')
        plt.subplot(1, 3, 3)
        plt.title('diff')
        mask_diff = np.logical_xor(mask1, mask2)
        plt.imshow(mask_diff, cmap='viridis', origin='lower')
        plt.show()

    temp = (np.where(~mask1, msr_sst[index, :, :] == msrc_sst[index, :, :], True))
    if not np.all(temp):
        plt.figure(figsize=(10, 8))
        plt.subplot(1, 3, 1)
        plt.title('sst of msr')
        plt.imshow(msr_sst[index, :, :], cmap='viridis', origin='lower')
        plt.subplot(1, 3, 2)
        plt.title('sst of msrc')
        plt.imshow(msrc_sst[index, :, :], cmap='viridis', origin='lower')
        plt.subplot(1, 3, 3)
        plt.title('diff')
        sst_diff = np.where(~mask1, np.abs(msr_sst[index, :, :] - msrc_sst[index, :, :]), np.nan)
        #print min and max of difference
        print('min of diff:', np.nanmin(sst_diff))
        print('max of diff:', np.nanmax(sst_diff))
        plt.imshow(sst_diff, cmap='viridis', origin='lower')
        plt.colorbar
        plt.show()

In [ ]:
#msr_sst = msr_sst.values
msr_sst_t_array = msr_sst_t.values
msrc_sst_t_array = msrc_sst_t.values

In [ ]:
#remove index 3235 and 3236 from the dataset
msr_sst = np.delete(msr_sst, [3235, 3236], axis=0)
msr_sst_t = np.delete(msr_sst_t, [3235, 3236], axis=0)

In [ ]:
# Check if the clouds in sst_t are always greater than the clouds in sst

for index in range(msr_sst.shape[0]):

    # MSR - Check if the clouds in sst_t are always greater than the clouds in sst
    mask1 = np.isnan(msr_sst[index, :, :])
    mask2 = np.isnan(msr_sst_t[index, :, :])
    temp = np.logical_or(~mask1, mask2) # if sst is masked, sst_t is also masked (sst_t masks more)     Vero anche se invertito: SST=SST_T per quanto riguarda i NaN
    if not np.all(temp):
        print('msr index:', index)  

#     # MSRC - Check if the clouds in sst_t are always greater than the clouds in sst
#     mask3 = np.isnan(msrc_sst[index, :, :])
#     mask4 = np.isnan(msrc_sst_t[index, :, :])
#     temp = np.logical_or(~mask3, mask4)
#     if not np.all(temp):
#         print('msrc index:', index)
    
#     # Check if the clouds in MSRC_sst_t are always greater than the clouds in MSR_sst_t
#     temp = np.logical_or(~mask2, mask4)
#     if not np.all(temp):
#         print('msr sst_t index:', index)
#     # Check if the clouds in MSR_sst_t are always greater than the clouds in MSRC_sst_t
#     temp = np.logical_or(~mask4, mask2)
#     if not np.all(temp):
#         print('msrc sst_t index:', index)   #FALSE!

# # Conclusion: MSRC_SST_T is the most clouded. the normal stt are the same in both datasets.

Comparison with our data

In [ ]:
med_coords = (-5, 36, 30, 46)  # (lon_min, lon_max, lat_min, lat_max)
dincae_coords = (12, 19, 40, 46)  # (lon_min, lon_max, lat_min, lat_max)
raw_coords = (310, 566, 0, 256)  # (x_min, x_max, y_min, y_max)

In [ ]:
# Example plot of a specific day for our data and dincae data

d_index = 22
msr_time = modis_sst_revlat['time']
print(msr_time[d_index].values)

ds = xr.open_dataset('dati/y2002_2004c/AQUA_MODIS.20030123.L3m.DAY.SST4.x_sst4.nc')    # Open the dataset
print(ds.attrs['time_coverage_end'])

data1 = ds['sst4']   # Extract the sst variables
data1 = data1.sel(lon=slice(dincae_coords[0], dincae_coords[1]), lat=slice(dincae_coords[3], dincae_coords[2]))
data2 = modis_sst_revlat['sst']

print(data1.shape, data2.shape)

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.imshow(data1)
plt.subplot(1, 2, 2)
plt.imshow(data2[d_index], origin='lower')
plt.show()

In [ ]:
# Extract lat and lon from our dataset, and raw coordinates from dincae data

example_ds = xr.open_dataset('dati/y2002_2004c/AQUA_MODIS.20030122.L3m.DAY.SST.x_sst.nc')    # Open the dataset
print(example_ds['sst'].shape)

# Calculate the resolution
lon_res = (med_coords[1] - med_coords[0]) / example_ds['sst'].shape[1]
lat_res = (med_coords[3] - med_coords[2]) / example_ds['sst'].shape[0]

# Convert the raw coordinates of our data ([0:256, 310:566]) to longitude and latitude
lon_min = med_coords[0] + raw_coords[0] * lon_res
lon_max = med_coords[0] + raw_coords[1] * lon_res
lat_min = med_coords[3] - raw_coords[3] * lat_res
lat_max = med_coords[3] - raw_coords[2] * lat_res

lonlat_coords = (lon_min, lon_max, lat_min, lat_max)    # (lon_min, lon_max, lat_min, lat_max)
print('lon/lat coords of italy square:', lonlat_coords)

# Convert the longitude and latitude of dincae data to raw coordinates
x_min = int((dincae_coords[0] - med_coords[0]) / lon_res)
x_max = int((dincae_coords[1] - med_coords[0]) / lon_res)
y_min = int((med_coords[3] - dincae_coords[3]) / lat_res)
y_max = int((med_coords[3] - dincae_coords[2]) / lat_res)

coords_on_med = (x_min, x_max, y_min, y_max)    # [0:144, 408:576]
print('xy coords for dincae on the mediterrean:', coords_on_med)
coords_on_italy = (x_min-raw_coords[0], x_max-raw_coords[0], y_min-raw_coords[2], y_max-raw_coords[2])  # [0:144, 98:266]
print('xy coords for dincae on italy:', coords_on_italy)

plt.figure(figsize=(15, 5))
# Cut dataset values using longitude and latitude
data_slice = example_ds['sst'].sel(lon=slice(lonlat_coords[0], lonlat_coords[1]), lat=slice(lonlat_coords[3], lonlat_coords[2]))
print(data_slice.shape)
plt.subplot(1, 3, 1)
plt.title('Italy cut using lon/lat')
plt.imshow(data_slice)

# Cut dincae values from dataset using raw coordinates
data_crop = example_ds['sst'].values[coords_on_med[2]:coords_on_med[3], coords_on_med[0]:coords_on_med[1]]
print(data_crop.shape)
plt.subplot(1, 3, 2)
plt.title('dincae cut using raw coordinates from mediterrean')
plt.imshow(data_crop)

# Cut dincae values from italy square using raw coordinates
data_crop_italy = example_ds['sst'].values[raw_coords[2]:raw_coords[3], raw_coords[0]:raw_coords[1]]
data_crop_italy = data_crop_italy[coords_on_italy[2]:coords_on_italy[3], coords_on_italy[0]:coords_on_italy[1]]
print(data_crop_italy.shape)
plt.subplot(1, 3, 3)
plt.title('dincae cut using raw coordinates from italy')
plt.imshow(data_crop_italy)

plt.show()

In [ ]:
# Apply the lonlat coordinates to obtain italy_mask, check if it's the same as the one obtained with raw coordinates

tmask = xr.open_dataset('dati/mist/tmask_interpolated.nc')

ocean_mask = tmask['tmask']   # Extract the sst data
ocean_mask = ocean_mask.sel(lat=slice(None, None, -1))   # Flip the latitude axis to match the data
print('ocean mask shape:', ocean_mask.shape)
# plt.imshow(ocean_mask)
# plt.show()

# Cut using raw coordinates
italy_mask1 = ocean_mask[0:256, 310:566]    # Focus on italy, 0 for land, 1 for sea
italy_mask1 = italy_mask1.astype(bool)
print('italy mask\'s shape cut with raw coordinates', italy_mask1.shape)
# plt.imshow(italy_mask1)
# plt.show()

# Cut using lonlat coordinates
italy_mask2 = ocean_mask.sel(lon=slice(lonlat_coords[0], lonlat_coords[1]), lat=slice(lonlat_coords[3], lonlat_coords[2]))
italy_mask2 = italy_mask2.astype(bool)
print('italy mask\'s shape cut with lat and lon', italy_mask2.shape)
# plt.imshow(italy_mask2)
# plt.show()

# Check if the two masks are the same
temp = (italy_mask1 == italy_mask2)
if np.all(temp):
    print('The two masks are the same')

In [ ]:
# Get Dincae Italy mask 

# Get dincae italy mask and check if it has the same proportions as the real dincae mask
dincae_italy_mask_m = ocean_mask.sel(lon=slice(dincae_coords[0], dincae_coords[1]), lat=slice(dincae_coords[3], dincae_coords[2]))
dincae_italy_mask_m = dincae_italy_mask_m.astype(bool)

italy_mask = ocean_mask[0:256, 310:566]
dincae_italy_mask_i = italy_mask[coords_on_italy[2]:coords_on_italy[3], coords_on_italy[0]:coords_on_italy[1]]  #[98:266, 0:144]
dincae_italy_mask_i = dincae_italy_mask_i.astype(bool)

print('msr mask shape:', msr_mask.shape, ', proportions:', msr_mask.shape[0]/msr_mask.shape[1])
print('dincae italy mask shape:', dincae_italy_mask_m.shape, ', proportions:', dincae_italy_mask_m.shape[0]/dincae_italy_mask_m.shape[1])
print('dincae italy mask shape:', dincae_italy_mask_i.shape, ', proportions:', dincae_italy_mask_i.shape[0]/dincae_italy_mask_i.shape[1], '(10 pixels on the side are cut off)')

# cut italy_mask to select dincae coordinates
subItalyMask = ocean_mask[y_min:y_max, x_min:x_max]
subItalyMask = subItalyMask.astype(bool)

#plt.imshow(dincae_italy_mask)

In [ ]:
# Compare masks

# Plot both masks
plt.figure(figsize=(16, 8))
plt.subplot(1, 3, 1)
plt.title('Actual Dincae Italy mask')
plt.imshow(msr_mask, origin='lower')    # CAREFUL! The actual mask is flipped
plt.subplot(1, 3, 2)
plt.title('Our Dincae Italy mask')
plt.imshow(dincae_italy_mask_m)
plt.subplot(1, 3, 3)
plt.title('Our Dincae Italy mask cut on italy_mask')
plt.imshow(dincae_italy_mask_i)
plt.show()

# Calculate difference and plot it
mask_diff = np.flipud(msr_mask) - dincae_italy_mask_m
plt.imshow(mask_diff, cmap='viridis')
plt.colorbar()

In [ ]:
#Create an appropriate dincae mask
dincae_mask = dincae_italy_mask_m.copy()

# Find the coordinates of the -1 value in mask_diff
coords = np.argwhere(mask_diff.values == -1)

# If there really are -1 values
if coords.size:
    for coord in coords:
        x, y = coord    # Get the coordinates
        dincae_mask[x, y] = 0 # Set the corresponding point in dincae_mask to 0

# Plot final mask
plt.imshow(dincae_mask, cmap='viridis')
plt.show()

#Check difference with real dincae mask
check_differences = np.flipud(msr_mask) - dincae_mask
plt.imshow(check_differences, cmap='viridis')
plt.colorbar()
plt.show()

Dataset Creation for 2003-2016 span

In [ ]:
#Dataset creation (2003-2016)

# Get a list of all .nc files in the directories and combine all the file lists into one list
files_y2002_2004c = glob.glob('dati/y2002_2004c/*.nc')
files_y2005_2009c = glob.glob('dati/y2005_2009c/*.nc')
files_y2010_2014c = glob.glob('dati/y2010_2014c/*.nc')
files_y2015_2019c = glob.glob('dati/y2015_2019c/*.nc')
#files_y2020_2023c = glob.glob('dati/y2020_2023c/*.nc')
all_files = files_y2002_2004c + files_y2005_2009c + files_y2010_2014c + files_y2015_2019c #+ files_y2020_2023c


# Datasets and dates lists, for day and night
dincae_dataset = []; dincae_date = []

# Define the start and end dates
start_date = pd.to_datetime('2003-01-01').date()
end_date = pd.to_datetime('2016-12-31').date()

for file in all_files:
    ds = xr.open_dataset(file)
    # Check if the dataset is night by checking the variable name: 'sst4' for night
    if 'sst4' in ds:
        # Extract the date from the product_name attribute
        date = pd.to_datetime(ds.attrs['product_name'].split('.')[1]).date()

        # Check if the date is within the desired range
        if start_date <= date <= end_date:
            data = ds['sst4'].values[0:256, 310:566]
            data = np.where(italy_mask, data, np.nan)    # Cut away measurements of lakes and rivers
            
            # Convert the date to an integer
            date = np.array(date, dtype='datetime64[D]')
            date = date.astype(int)
            
            dincae_dataset.append(data)
            dincae_date.append(date)

In [ ]:
# Convert the lists to a numpy array

dincae_dataset = np.array(dincae_dataset)
dincae_date = np.array(dincae_date)

print(dincae_dataset.shape)
print(dincae_date.shape)
print(dincae_date)

In [ ]:
# Save the msr datasets in numpy arrays
msr_sst_t_array = modis_sst_revlat['sst_t'].values
msrc_sst_t_array = modis_sst_revlat_add_clouds['sst_t'].values
print(msr_sst_t_array.shape)
print(msrc_sst_t_array.shape)

In [ ]:
# Check for missing days in our dataset

# Convert the dincae time stamps to int like the ones in our dataset
dincae_time = modis_sst_revlat['time']
#Consider only the date part, and convert dates to epoch time
dincae_time = np.array(dincae_time.values, dtype='datetime64[D]')
dincae_time = dincae_time.astype(int)
print('time is now in the form of:', dincae_time)

# #reconvert to datetime
# dincae_time = np.array(dincae_time, dtype='datetime64[D]')
# print(dincae_time)
# # Check if the time stamps are the same
# time_diff = msr_time.values - dincae_time
# print('time_diff:', time_diff)

# #Some values are missing in our dataset! Identify them and remove them from the msr arrays
# missing_values = np.setdiff1d(dincae_time, dincae_date)
# dates_missing_values = np.array(missing_values, dtype='datetime64[D]')
# print('missing dates:', missing_values, ', aka', dates_missing_values)


# #Remove the missing values from the msr arrays
# mask = np.isin(dincae_time, dincae_date)    # Create a boolean mask and apply it to the msr arrays
# msr_sst_array = msr_sst_array[mask]
# msrc_sst_t_array = msrc_sst_t_array[mask]

In [ ]:
#find the date 15288 and 15289 in dincae_time
index1 = np.where(dincae_time == 15288)[0][0]
index2 = np.where(dincae_time == 15289)[0][0]
print('index1:', index1)
print('index2:', index2)

# plot the msr sst and msrc sst_t for these 2 dates
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.title('msr sst')
plt.imshow(msr_sst_array[index1], cmap='viridis')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.title('msrc sst_t')
plt.imshow(msrc_sst_t_array[index1], cmap='viridis')
plt.colorbar()
plt.show()
#print min and max of index1
print('min of msr sst:', np.nanmin(msr_sst_array[index1]))
print('max of msr sst:', np.nanmax(msr_sst_array[index1]))
print('min of msrc sst_t:', np.nanmin(msrc_sst_t_array[index1]))
print('max of msrc sst_t:', np.nanmax(msrc_sst_t_array[index1]))

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.title('msr sst')
plt.imshow(msr_sst_array[index2], cmap='viridis')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.title('msrc sst_t')
plt.imshow(msrc_sst_t_array[index2], cmap='viridis')
plt.colorbar()
plt.show()
#print min and max of index2
print('min of msr sst:', np.nanmin(msr_sst_array[index2]))
print('max of msr sst:', np.nanmax(msr_sst_array[index2]))
print('min of msrc sst_t:', np.nanmin(msrc_sst_t_array[index2]))
print('max of msrc sst_t:', np.nanmax(msrc_sst_t_array[index2]))

In [ ]:
#change the msr_sst_array in indexes [3235, 3236] so that they have nans in points where they have 45.000717 degrees

indexes_to_change = [3235, 3236]
for index in indexes_to_change:
    msr_sst_array[index] = np.where(msr_sst_array[index] == 45.000717, np.nan, msr_sst_array[index])

In [ ]:
#find in msr_sst_array and msrc_sst_t_array the measurements that have at least one point at 45.000717 degrees, and plot them
#find the indexes of the points that have at least one point at 45.000717 degrees
indexes = []
for i in range(len(msr_sst_array)):
    if 45.000717 in msr_sst_array[i]:
        indexes.append(i)
print('indexes:', indexes)

#plot the measurements that have at least one point at 45.000717 degrees
for idx in indexes:
    plt.figure(figsize=(16, 8))
    plt.subplot(1, 2, 1)
    plt.title(f'msr sst at index {idx}')
    plt.imshow(msr_sst_array[idx], cmap='viridis')
    plt.colorbar()
    plt.subplot(1, 2, 2)
    plt.title(f'msrc sst_t at index {idx}')
    plt.imshow(msrc_sst_t_array[idx], cmap='viridis')
    plt.colorbar()
    plt.show()

In [ ]:
# Now the msr arrays have the same dates as our dataset
print(msr_sst_t_array.shape)
print(msrc_sst_t_array.shape)
# print(dincae_dataset.shape)
# print(dincae_date.shape)


plt.imshow(msr_sst_t_array[22])
plt.show()

# Msr arrays are flipped, so flip vertically (flip on axis=1)
msr_sst_t_array = np.flip(msr_sst_t_array, axis=1)
msrc_sst_t_array = np.flip(msrc_sst_t_array, axis=1)

plt.imshow(msr_sst_t_array[22])
plt.show()

In [ ]:
# Execute to cut off days with no added clouds in the dincae dataset

same_mask_indices = []

for i in range(msrc_sst_t_array.shape[0]):
    cloud_mask = np.logical_not(np.isnan(msrc_sst_t_array[i]))
    clear_mask = np.logical_not(np.isnan(msr_sst_array[i]))

    # Check if the masks are the same
    if np.all(cloud_mask == clear_mask):
        same_mask_indices.append(i)
    
# Convert the list of indices to a numpy array
same_mask_indices = np.array(same_mask_indices)
# Create boolean arrays that are True for indices that should be kept
keep_indices = np.ones(msrc_sst_t_array.shape[0], dtype=bool)
keep_indices[same_mask_indices] = False

msrc_sst_t_array = msrc_sst_t_array[keep_indices]
msr_sst_array = msr_sst_array[keep_indices]

print('shape of new msrc_sst_t_array:', msrc_sst_t_array.shape)
print('shape of new msr_sst_array:', msr_sst_array.shape)

In [ ]:
# CREATE AND APPLY VALIDATION MASK

tmask = xr.open_dataset('dati/mist/tmask_interpolated.nc')
ocean_mask = tmask['tmask']   # Extract the sst data
ocean_mask = ocean_mask.sel(lat=slice(None, None, -1))   # Flip the latitude axis to match the data

ditalymask = ocean_mask.sel(lon=slice(dincae_coords[0], dincae_coords[1]), lat=slice(dincae_coords[3], dincae_coords[2]))
ditalymask = ditalymask.astype(bool)

plt.imshow(ditalymask)
plt.show()
print('ditalymask shape:', ditalymask.shape)

# plt.imshow(msr_sst_t_array[22])
# plt.show()

# for each measurement in msr_sst_array and msrc_sst_t_array, filter using ditalymask
# if the measurement is not in the ocean, set it to nan
for i in range(msr_sst_t_array.shape[0]):
    msr_sst_t_array[i] = np.where(ditalymask, msr_sst_t_array[i], np.nan)
    msrc_sst_t_array[i] = np.where(ditalymask, msrc_sst_t_array[i], np.nan)

plt.imshow(msr_sst_t_array[22])
plt.show()

In [ ]:
# Save the arrays to a file
np.save('dati/mist/dincae/msr_sst_t_array.npy', msr_sst_t_array)
np.save('dati/mist/dincae/msrc_sst_t_array.npy', msrc_sst_t_array)

# np.save('dati/mist/dincae/dincae_dataset.npy', dincae_dataset)
# np.save('dati/mist/dincae/dincae_date.npy', dincae_date)

Confrontation

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

import glob
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, Conv2DTranspose, Concatenate, concatenate, AveragePooling2D, UpSampling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
def customLoss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)

    real_mask = y_true[:,:,:,1:2]        # 0 for land/clouds, 1 for sea
    y_true = y_true[:,:,:,0:1]          # The true SST values. Obfuscated areas are already converted to 0
    
    # Calculate the squared error only over clear sea
    squared_error = tf.square(y_true - y_pred)
    masked_error = squared_error * real_mask

    # Calculate the mean of the masked errors
    clear_loss = tf.reduce_mean(masked_error)     # The final loss

    return clear_loss

In [ ]:
def ClearMetric(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)

    art_mask = y_true[:,:,:,2:3]   # 0 for land/clouds + artificials, 1 for clear, untouched sea
    y_true = y_true[:,:,:,0:1]  # The true SST values. Obfuscated areas are already converted to 0

    # Calculate the squared error only over clear sea
    squared_error = tf.square(y_true - y_pred)
    clear_masked_error = squared_error * art_mask
    # Calculate the mean of the masked errors
    clr_metric = tf.reduce_sum(clear_masked_error) / tf.reduce_sum(art_mask)

    #loss = clear_loss
    return clr_metric

In [ ]:
def ArtificialMetric(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)    # Was getting an error because of the different types: y_true in the metrics is float64 instead of the normal float32

    real_mask = y_true[:,:,:,1:2]  # 0 for land/clouds, 1 for clear sea
    art_mask = y_true[:,:,:,2:3]  # 0 for land/clouds + artificials, 1 for clear sea
    added_mask = real_mask - art_mask  # 1 only for hidden sea, 0 for land/clouds and visible sea
    y_true = y_true[:,:,:,0:1]  # The true SST values. Obfuscated areas are already converted to 0

    # Calculate the squared error only over artificially clouded areas
    squared_error = tf.square(y_true - y_pred)
    artificial_masked_error = squared_error * added_mask
    # Calculate the mean of the masked errors
    art_metric = tf.reduce_sum(artificial_masked_error) / tf.reduce_sum(added_mask)

    return art_metric

In [ ]:
batch_size = 32
input_shape = (256, 256, 4)
lr = 1e-4
loss = customLoss
metrics = [ClearMetric, ArtificialMetric]

italy_mask = np.load('dati/mist/datasets/italy_mask.npy')
training_baseline_n = np.load('dati/mist/datasets/training_baseline_n.npy')

dincae_date = np.arange(12053, 17167)   # 2003-01-01 to 2016-12-31

msr_sst_array = np.load('dati/mist/dincae/msr_sst_array.npy')
msrc_sst_t_array = np.load('dati/mist/dincae/msrc_sst_t_array.npy')

data_min = np.nanmin(msr_sst_array, axis=(1, 2))
data_max = np.nanmax(msr_sst_array, axis=(1, 2))
data_min = np.nanmin(data_min)
data_max = np.nanmax(data_max)
print('data_min:', data_min, ', data_max:', data_max)

norm_msr_sst_array = 2 * ((msr_sst_array - data_min) / (data_max - data_min)) - 1
norm_msrc_sst_t_array = 2 * ((msrc_sst_t_array - data_min) / (data_max - data_min)) - 1

In [ ]:
offset_y = 0    # 144 pixels tall
offset_x = 98   # since x is 168 pixels wide but it has to stop at 256, 10 pixels are cut off from the right
end_y = 144
end_x = 256

In [ ]:
# Dincae Generator
# Dincae always uses the night dataset
def dincae_generator(batch_size, msrData, msrcData, date):
    i = 0   # Counter for the dataset. We will use the whole dataset, one batch at a time
    while True:
        batch_x = np.zeros((batch_size, 256, 256, 4))
        batch_y = np.zeros((batch_size, 256, 256, 2))
        batch_dates = np.empty(batch_size)

        for b in range(batch_size):
            # Get a day from msr dataset, and extract the corresponding mask from msrc dataset. Cut to the right size.
            msr_image = np.nan_to_num(msrData[i], nan=0)
            msr_image = msr_image[:end_y-offset_y, :end_x-offset_x]
            msr_mask = np.isnan(msrData[i])
            msr_mask = msr_mask[:end_y-offset_y, :end_x-offset_x]

            msrc_mask = np.isnan(msrcData[i])
            msrc_mask = msrc_mask[:end_y-offset_y, :end_x-offset_x]
            
            # Create a 256x256 image and masks, and insert them in the right place
            greater_msr_image = np.zeros((256, 256), dtype=float)
            greater_msr_image[offset_y:end_y, offset_x:end_x] = msr_image
            greater_msr_mask = np.ones((256, 256), dtype=bool)
            greater_msr_mask[offset_y:end_y, offset_x:end_x] = msr_mask
            greater_msrc_mask = np.ones((256, 256), dtype=bool)
            greater_msrc_mask[offset_y:end_y, offset_x:end_x] = msrc_mask
            
            # Apply the mask to the image
            image_masked = np.where(greater_msrc_mask, 0, greater_msr_image)

            # Extract the day-of-year from the date
            date_series = pd.to_datetime(date[i], unit='D', origin=pd.Timestamp('1970-01-01'))
            day_of_year = date_series.dayofyear

            # Fix masks before they are used in the loss and metric functions
            greater_msrc_mask = np.logical_not(greater_msrc_mask) # 1 for clear sea, 0 for land/clouds + artificials
            greater_msr_mask = np.logical_not(greater_msr_mask) # 1 for clear sea, 0 for land/clouds

            # Create batch_x and batch_y
            batch_x[b, ..., 0] = image_masked                           #artificially cloudy image
            batch_x[b, ..., 1] = greater_msrc_mask                      #artificial mask
            batch_x[b, ..., 2] = italy_mask                             #land-sea mask
            batch_x[b, ..., 3] = training_baseline_n[day_of_year - 1]   #baseline values for the current day (day_of_year starts from 1)

            batch_y[b, ..., 0] = greater_msr_image                      #real image
            batch_y[b, ..., 1] = greater_msr_mask                       #real mask

            # Add the date information to the batch
            batch_dates[b] = date[i]

            # Increment the index
            i += 1
            if i >= msrData.shape[0]:
                i = 0
        
        yield batch_x, batch_y, batch_dates

In [ ]:
#dincae_test_gen = dincae_generator(batch_size, dincae_dataset, dincae_date)
dincae_test_gen = dincae_generator(batch_size, norm_msr_sst_array, norm_msrc_sst_t_array, dincae_date)

In [ ]:
# Test the dincae generator

x,y,dates = next(dincae_test_gen)
r = np.random.randint(0, x.shape[0])    # Choose a random image from the batch

maskdiff = y[r, :, :, 1] - x[r, :, :, 1]
plt.imshow(maskdiff, cmap='viridis')
plt.colorbar()

# Plot the image
plt.figure(figsize=(15, 10))

# Plot the y data
plt.subplot(2, 2, 1)
plt.imshow(y[r, :, :, 0], cmap='viridis')
plt.title("msr data)")
plt.colorbar()

# Plot the x data
plt.subplot(2, 2, 2)
plt.imshow(x[r, :, :, 0], cmap='viridis')
plt.title("covered msr / msrc data (model input)")
plt.colorbar()

# Plot the mask used as input
plt.subplot(2, 2, 3)
plt.imshow(y[r, :, :, 1], cmap='viridis')
plt.title("msr mask")

# Plot the baseline values
plt.subplot(2, 2, 4)
plt.imshow(x[r, :, :, 1], cmap='viridis')
plt.title("msrc mask (model input)")

plt.show()

# Information about the data
print("are there nans?", np.isnan(x).any())
print("x.shape:", x.shape)
print("y.shape:", y.shape)
print("min of all x:", np.min(x[..., 0]))
print("max of all x:", np.max(x[..., 0]))
print("min of this x:", np.min(x[r, :, :, 0]))
print("max of this x:", np.max(x[r, :, :, 0]))

date = pd.to_datetime(dates[r], unit='D', origin=pd.Timestamp('1970-01-01')).date()
print("date of this image:", date, "(", dates[r], ")")

In [ ]:
# %%
# U-Net model with residual blocks

def ResidualBlock(depth):
    def apply(x):
        input_depth = x.shape[3]    # Get the number of channels from the channels dimension
        if input_depth == depth:    # It's already the desired channel number
            residual = x
        else:                       # Adjust the number of channels with a 1x1 convolution
            residual = Conv2D(depth, kernel_size=1)(x)

        x = BatchNormalization(center=False, scale=False)(x)    
        x = Conv2D(depth, kernel_size=3, padding="same", activation='swish')(x) 
        x = Conv2D(depth, kernel_size=3, padding="same")(x)
        x = Add()([x, residual])
        return x
    
    return apply


def DownBlock(depth, block_depth):
    def apply(x):
        x, skips = x
        for _ in range(block_depth):
            x = ResidualBlock(depth)(x)
            skips.append(x)
        x = AveragePooling2D(pool_size=2)(x)    #downsampling
        return x

    return apply


def UpBlock(depth, block_depth):
    def apply(x):
        x, skips = x
        x = UpSampling2D(size=2, interpolation="bilinear")(x)   #upsampling
        for _ in range(block_depth):
            x = Concatenate()([x, skips.pop()])
            x = ResidualBlock(depth)(x)
        return x

    return apply


def get_Unet(image_size, depths, block_depth):
    input_images = Input(shape=image_size)  #input layer
    
    x = Conv2D(depths[0], kernel_size=1)(input_images)  #reduce the number of channels

    skips = []  #store the skip connections
    
    for depth in depths[:-1]:   #downsampling layers
        x = DownBlock(depth, block_depth)([x, skips])

    for _ in range(block_depth):    #middle layer
        x = ResidualBlock(depths[-1])(x)

    for depth in reversed(depths[:-1]):   #upsampling layers
        x = UpBlock(depth, block_depth)([x, skips])

    x = Conv2D(1, kernel_size=1, kernel_initializer="zeros", name = "output_noise")(x)  #output layer
    
    return Model(input_images, outputs=x, name="UNetInpainter")

In [ ]:
# %%
# Define the model
depths = [32, 64, 128]
block_depth = 2
model = get_Unet(input_shape, depths, block_depth)
#model.summary()

In [ ]:
# %%
# Compile model with custom loss function
opt = Adam(learning_rate=lr)
model.compile(optimizer=opt, loss=loss, metrics=[metrics])

In [ ]:
#LOAD WEIGHTS
model.load_weights('weights/standard.h5')

# WARNING: the 'baseline.h5' weight is trained on normalized data, but dincae uses the original data.
# Need to train a new unet on the original data and use that weight
# The use of absolute data may also be a problem for the generator?

In [ ]:
# COMPUTE ERRORS
print("Evaluating the errors, please wait...")
print("Ignore runtime errors, they are caused by 'nan slices' in the data")

# Initialize lists to store the errors and the maximum errors
all_errors = []
max_errors = []         #for each element
clear_errors = []
clear_max_errors = []   #for each element always visible
hidden_errors = []
hidden_max_errors = []  #for each element hidden artificially

# Iterate over all the dataset
for i in range(len(norm_msr_sst_array) // batch_size):
    if i < len(norm_msr_sst_array) // batch_size:
        x_true, y_true, dates = next(dincae_test_gen)
    else:
        # Special case for the last batch
        remaining_size = len(norm_msr_sst_array) % batch_size
        if remaining_size == 0:
            break
        x_true, y_true, dates = next(dincae_test_gen)
        x_true = x_true[:remaining_size]
        y_true = y_true[:remaining_size]
        dates = dates[:remaining_size]

    predictions = model.predict(x_true, verbose=0)  # Prediction

    # Denormalize
    predictions_adriatic = predictions[:, offset_y:end_y, offset_x:end_x, 0]
    msr_images_adriatic = y_true[:, offset_y:end_y, offset_x:end_x, 0]
    predictions_denorm = ((predictions_adriatic + 1) / 2) * (data_max - data_min) + data_min  # Denormalize the data
    msr_images_denorm = ((msr_images_adriatic + 1) / 2) * (data_max - data_min) + data_min  # Denormalize the data

    # Get the masks and calculate the errors
    realMask = y_true[:, offset_y:end_y, offset_x:end_x, 1]     #real mask
    hiddenMask = np.not_equal(y_true[:, offset_y:end_y, offset_x:end_x, 1], x_true[:, offset_y:end_y, offset_x:end_x, 1])   #hidden sea
    clearMask = x_true[:, offset_y:end_y, offset_x:end_x, 1]                                  #clear sea


    errors = np.where(realMask, np.abs(predictions_denorm - msr_images_denorm), np.nan)
    clear_errors_batch = np.where(clearMask, np.abs(predictions_denorm - msr_images_denorm), np.nan)
    hidden_errors_batch = np.where(hiddenMask, np.abs(predictions_denorm - msr_images_denorm), np.nan)

    # Flatten the errors and add to the list
    all_errors.extend(errors.flatten())
    max_errors.append(np.nanmax(errors))
    clear_errors.extend(clear_errors_batch.flatten())
    clear_max_errors.append(np.nanmax(clear_errors_batch))
    hidden_errors.extend(hidden_errors_batch.flatten())
    hidden_max_errors.append(np.nanmax(hidden_errors_batch))

# Convert to numpy array for easier calculations
all_errors = np.array(all_errors)
max_errors = np.array(max_errors)
clear_errors = np.array(clear_errors)
clear_max_errors = np.array(clear_max_errors)
hidden_errors = np.array(hidden_errors)
hidden_max_errors = np.array(hidden_max_errors)

# Calculate the metrics over all errors
avg_error = np.nanmean(all_errors)
max_error = np.nanmax(all_errors)
avg_max_error = np.nanmean(max_errors)
var_error = np.nanvar(all_errors)
rmse_error = np.sqrt(np.nanmean(all_errors**2))
# Calculate the metrics over clear errors
avg_clear_error = np.nanmean(clear_errors)
max_clear_error = np.nanmax(clear_errors)
avg_max_clear_error = np.nanmean(clear_max_errors)
var_clear_error = np.nanvar(clear_errors)
rmse_clear_error = np.sqrt(np.nanmean(clear_errors**2))
# Calculate the metrics over hidden errors
avg_hidden_error = np.nanmean(hidden_errors)
max_hidden_error = np.nanmax(hidden_errors)
avg_max_hidden_error = np.nanmean(hidden_max_errors)
var_hidden_error = np.nanvar(hidden_errors)
rmse_hidden_error = np.sqrt(np.nanmean(hidden_errors**2))

# Print the metrics calculated over all elements
print(f"\nAverage error over all elements:", avg_error)
print(f"Average maximum error:", avg_max_error, ", and maximum error:", max_error)
print(f"Variance of errors over all elements:", var_error)
print(f"RMSE over all elements:", rmse_error)

# Print the metrics calculated over clear elements
print(f"\nAverage error over clear elements:", avg_clear_error)
print(f"Average maximum error over clear elements:", avg_max_clear_error, ", and maximum error over clear elements:", max_clear_error)
print(f"Variance of errors over clear elements:", var_clear_error)
print(f"RMSE over clear elements:", rmse_clear_error)

# Print the metrics calculated over hidden elements
print(f"\nAverage error over hidden elements:", avg_hidden_error)
print(f"Average maximum error over hidden elements:", avg_max_hidden_error, ", and maximum error over hidden elements:", max_hidden_error)
print(f"Variance of errors over hidden elements:", var_hidden_error)
print(f"RMSE over hidden elements:", rmse_hidden_error)

In [ ]:
# MODIFIED TEST - identify high-error measurements

# Initialize a list to store the dates, predictions, true values, and errors associated with high-error measurements
high_error_dates = []
high_error_predictions = []
high_error_true_values = []
high_error_data = []

# Set the error threshold
error_threshold = 10

# Generate a batch
for i in range(len(norm_msr_sst_array) // batch_size):
    if i < len(norm_msr_sst_array) // batch_size:
        x_true, y_true, dates = next(dincae_test_gen)
    else:
        # Special case for the last batch
        remaining_size = len(norm_msr_sst_array) % batch_size
        if remaining_size == 0:
            break
        x_true, y_true, dates = next(dincae_test_gen)
        x_true = x_true[:remaining_size]
        y_true = y_true[:remaining_size]
        dates = dates[:remaining_size]

    predictions = model.predict(x_true, verbose=0)  # Prediction

    # Get the mask to select only the interested subsections
    clear_masks = x_true[..., 1]                                            #unclouded
    hidden_sea_masks = np.not_equal(y_true[..., 1], x_true[..., 1])         #hidden sea
    clear_masks = clear_masks[:, offset_y:end_y, offset_x:end_x]
    hidden_sea_masks = hidden_sea_masks[:, offset_y:end_y, offset_x:end_x]

    # Evaluate the prediction compared to the msr image (compare only on the interested subsection)
    predictions_adriatic = predictions[:, offset_y:end_y, offset_x:end_x, 0]
    predictions_denorm = np.where(hidden_sea_masks, ((predictions_adriatic + 1) / 2) * (data_max - data_min) + data_min, np.nan)
    msr_images_adriatic = y_true[:, offset_y:end_y, offset_x:end_x, 0]
    msr_images_denorm = np.where(hidden_sea_masks, ((msr_images_adriatic + 1) / 2) * (data_max - data_min) + data_min, np.nan)

    # Calculate the errors
    h_errors = np.where(hidden_sea_masks, np.abs(predictions_denorm - msr_images_denorm), np.nan)


    # Check each day in the batch
    for j in range(h_errors.shape[0]):
        # If the maximum error for this day is above the threshold, store the date, prediction, true value, and error
        if np.nanmax(h_errors[j]) > error_threshold:
            high_error_dates.append(dates[j])
            high_error_predictions.append(predictions_denorm[j])
            high_error_true_values.append(msr_images_denorm[j])
            high_error_data.append(h_errors[j])
            

# Now, you can plot the images, predictions, errors, and dates of the high-error measurements
for i in range(len(high_error_dates)):
    print(f"Date: {high_error_dates[i]}")
    max_error = np.nanmax(high_error_data[i])
    print(f"Max Error: {max_error}")
    
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(high_error_true_values[i], cmap='hot', vmin=data_min, vmax=data_max)
    plt.title('True Value')
    plt.colorbar()
    plt.subplot(1, 3, 2)
    plt.imshow(high_error_predictions[i], cmap='hot', vmin=data_min, vmax=data_max)
    plt.title('Prediction')
    plt.colorbar()
    plt.subplot(1, 3, 3)
    plt.imshow(high_error_data[i], cmap='jet')
    plt.title('Error')
    plt.colorbar()
    plt.show()

In [ ]:
# TO REDO - Check for problematic spots in real images

# Generate a batch
x_true, y_true, dates = next(dincae_test_gen)
# predictions = model.predict(x_true, verbose=0)  # Prediction

# Denormalize
# predictions_adriatic = predictions[:, offset_y:end_y, offset_x:end_x, 0]
# predictions_denorm = ((predictions_adriatic + 1) / 2) * (data_max - data_min) + data_min  # Denormalize the data
msr_images_adriatic = y_true[:, offset_y:end_y, offset_x:end_x, 0]
true_values_denorm = ((msr_images_adriatic + 1) / 2) * (data_max - data_min) + data_min  # Denormalize the data

# Get the clear mask
# clearMask = y_true[..., 1]  # 1 for all clear sea, 0 for land/clouds

# Get the masks and calculate the errors
realMask = y_true[:, offset_y:end_y, offset_x:end_x, 1]     #real mask
hiddenMask = np.not_equal(y_true[:, offset_y:end_y, offset_x:end_x, 1], x_true[:, offset_y:end_y, offset_x:end_x, 1])   #hidden sea
clearMask = x_true[:, offset_y:end_y, offset_x:end_x, 1]                                  #clear sea

# # Initialize an array to store the differences
# errors = np.where(realMask, np.abs(predictions_denorm - true_values_denorm), np.nan)
# clear_errors_batch = np.where(clearMask, np.abs(predictions_denorm - true_values_denorm), np.nan)
# hidden_errors_batch = np.where(hiddenMask, np.abs(predictions_denorm - true_values_denorm), np.nan)

# For each image in the batch
for i in range(batch_size):
    problematic_spots = []
    # problematic_spot_found = False

    # For each pixel in the image
    for y in range(true_values_denorm.shape[0]):
        for x in range(true_values_denorm.shape[1]):
            if realMask[i, y, x] != 0: # Only consider the pixel if it's not masked
                pixel = true_values_denorm[i, y, x]

                # Get the values of the neighbors
                neighbors_mask = realMask[i, max(0, y-1):min(y+2, true_values_denorm.shape[0]), max(0, x-1):min(x+2, true_values_denorm.shape[1])]
                neighbors_values = true_values_denorm[i, max(0, y-1):min(y+2, true_values_denorm.shape[0]), max(0, x-1):min(x+2, true_values_denorm.shape[1])]
                neighbors_values[neighbors_mask == 0] = np.nan  # Replace masked values with np.nan

                # Calculate the max difference
                max_diff = np.nanmax(np.abs(neighbors_values - pixel)) if np.count_nonzero(~np.isnan(neighbors_values)) > 0 else 0

                # If the max difference is greater than a tot amount, it's a problematic spot
                if max_diff > 5:
                    print("Sample n°", i+1, ", date:", dates[i])
                    print(f"Problematic spot found in position ({y}, {x}), max difference: {max_diff}")
                    print("Adjacent values:")
                    print(neighbors_values)

                    # Set the flag to True and store the coordinates of the problematic spot
                    # problematic_spot_found = True
                    problematic_spots.append((x, y))

    # #If a problematic spot was found, plot the error image and the error image with the problematic spots marked
    # if problematic_spot_found:
    #     fig, axs = plt.subplots(1, 2, figsize=(24, 12))
    #     # Plot the error image
    #     axs[0].imshow(errors[i], cmap='jet')
    #     axs[0].set_title(f"Error map - Batch {i+1}")
    #     # Plot the error image with the problematic spots marked
    #     axs[1].imshow(errors[i], cmap='jet')
    #     axs[1].scatter(*zip(*problematic_spots), color='magenta')  # Scatter plot of the problematic spots
    #     axs[1].set_title(f"Error map - Batch {i+1} (Problematic spots marked)")

    #     plt.show()

In [ ]:
# ITERATION OVER DATASET - Check for problematic spots in real images

def calculate_spatial_variation(sample, threshold):
    """
    Calculate the spatial variation for each pixel by comparing it with its neighbors
    and return the average spatial variation for the sample.
    """
    height, width = sample.shape
    spatial_variations = np.zeros((height, width))
    max_variation_in_sample = 0
    
    for y in range(height):
        for x in range(width):
            pixel = sample[y, x]
            neighbors_values = sample[max(0, y-1):min(y+2, height), max(0, x-1):min(x+2, width)]
            max_diff = np.nanmax(np.abs(neighbors_values - pixel)) if np.count_nonzero(~np.isnan(neighbors_values)) > 0 else 0
            spatial_variations[y, x] = max_diff
            
            if max_diff > threshold:  # input threshold for high spatial variation
                print(f"High spatial variation found in position ({y}, {x}), max difference: {max_diff}")
            if max_diff > max_variation_in_sample:
                max_variation_in_sample = max_diff

    # Calculate the average spatial variation for the sample
    average_spatial_variation = np.nanmean(spatial_variations)
    return average_spatial_variation, max_variation_in_sample

treshold = 5                        # Threshold considered high spatial variation
average_spatial_variations = []
samples_with_high_variation = 0
highest_spatial_variation = 0
dataset = msr_sst_array   # Dataset to analyze

# Iterate over each sample in the dataset
for i in range(dataset.shape[0]):
    #print(f"Analyzing sample {i+1}")
    average_variation, max_variation_in_sample = calculate_spatial_variation(dataset[i, :, :], treshold)
    average_spatial_variations.append(average_variation)
    #print(f"Average spatial variation for sample {i+1}: {average_variation}")
    
    if max_variation_in_sample > treshold:  # Threshold for high spatial variation
        samples_with_high_variation += 1
    if max_variation_in_sample > highest_spatial_variation:
        highest_spatial_variation = max_variation_in_sample

# Calculate the overall average spatial variation
overall_average_variation = np.mean(average_spatial_variations)
print(f"Overall average spatial variation: {overall_average_variation}")
print(f"Number of samples with high spatial variation: {samples_with_high_variation}")
print(f"Highest spatial variation in dataset: {highest_spatial_variation}")

Check if erroneous dates in dincae data (11/11/10 and 11/11/11) are the same in the real TERRA data

In [ ]:
dincae_coords = (12, 19, 40, 46)  # (lon_min, lon_max, lat_min, lat_max)
#med_coords = (-5, 36, 30, 46)  # (lon_min, lon_max, lat_min, lat_max)

terra10 = xr.open_dataset('dati/mist/old/dincae/TERRA_MODIS.20111110.L3m.DAY.SST4.sst4.4km.nc')
#terra10
terra10 = terra10['sst4']
d10 = terra10.sel(lon=slice(dincae_coords[0], dincae_coords[1]), lat=slice(dincae_coords[3], dincae_coords[2]))
d10 = d10.values
print(d10.shape)

terra11 = xr.open_dataset('dati/mist/old/dincae/TERRA_MODIS.20111111.L3m.DAY.SST4.sst4.4km.nc')
#terra11
terra11 = terra11['sst4']
d11 = terra11.sel(lon=slice(dincae_coords[0], dincae_coords[1]), lat=slice(dincae_coords[3], dincae_coords[2]))
d11 = d11.values
print(d11.shape)

terra12 = xr.open_dataset('dati/mist/old/dincae/TERRA_MODIS.20111112.L3m.DAY.SST4.sst4.4km.nc')
#terra12
terra12 = terra12['sst4']
d12 = terra12.sel(lon=slice(dincae_coords[0], dincae_coords[1]), lat=slice(dincae_coords[3], dincae_coords[2]))
d12 = d12.values
print(d12.shape)

In [ ]:
#plot the sst4
plt.figure(figsize=(10, 8))

# vmin = min(np.nanmin(msrc_sst[3235]), np.nanmin(msrc_sst[3236]), np.nanmin(d10), np.nanmin(d11))
# vmax = max(np.nanmax(msrc_sst[3235]), np.nanmax(msrc_sst[3236]), np.nanmax(d10), np.nanmax(d11))
vmin = min(np.nanmin(d10), np.nanmin(d11))
vmax = max(np.nanmax(d10), np.nanmax(d11))

plt.subplot(2, 2, 1)
plt.title('Dincae - 2011-11-10')
plt.imshow(msrc_sst[3235], origin='lower', vmin=vmin, vmax=vmax)
plt.subplot(2, 2, 2)
plt.title('Terra MODIS (night) - 2011-11-10')
plt.imshow(d10, vmin=vmin, vmax=vmax)

plt.subplot(2, 2, 3)
plt.title('Dincae - 2011-11-11')
plt.imshow(msrc_sst[3236], origin='lower', vmin=vmin, vmax=vmax)
plt.subplot(2, 2, 4)
plt.title('Terra MODIS (night) - 2011-11-11')
plt.imshow(d11, vmin=vmin, vmax=vmax)

plt.show()

In [ ]:
#plot the sst4
plt.figure(figsize=(10, 4))
vmin = min(np.nanmin(d12), np.nanmin(msrc_sst[3237]))
vmax = max(np.nanmax(d12), np.nanmax(msrc_sst[3237]))

plt.subplot(1, 2, 1)
plt.title('Dincae - 2011-11-12')
plt.imshow(msrc_sst[3237], origin='lower', vmin=vmin, vmax=vmax)
plt.subplot(1, 2, 2)
plt.title('Terra MODIS (night) - 2011-11-12')
plt.imshow(d12, vmin=vmin, vmax=vmax)

plt.show()

In [ ]:
# Convert DataArray to numpy array and flatten
msrc_sst_flat = msrc_sst[3237].values
msrc_sst_flat = np.flip(msrc_sst_flat, axis=0)
msrc_sst_flat = msrc_sst_flat.flatten()
d12_flat = d12.flatten()

# Create masks for NaN values
mask_msrc_sst = np.isnan(msrc_sst_flat)
mask_d12 = np.isnan(d12_flat)

# Create masks for non-NaN values
non_nan_mask_msrc_sst = ~mask_msrc_sst
non_nan_mask_d12 = ~mask_d12

# Create a mask for points where both matrices have non-NaN values
non_nan_mask = non_nan_mask_msrc_sst & non_nan_mask_d12

# Compare the non-NaN values in the two matrices
are_values_equal = np.array_equal(msrc_sst_flat[non_nan_mask], d12_flat[non_nan_mask])

print("The non-NaN values in the two matrices are equal:", are_values_equal)

Confront with terra

In [ ]:
med_coords = (-5, 36, 30, 46)  # (lon_min, lon_max, lat_min, lat_max)
dincae_coords = (12, 19, 40, 46)  # (lon_min, lon_max, lat_min, lat_max)

# tmask = xr.open_dataset('dati/mist/tmask_interpolated.nc')
# ocean_mask = tmask['tmask']   # Extract the sst data
# ocean_mask = ocean_mask.sel(lat=slice(None, None, -1))   # Flip the latitude axis to match the data
# ditalymask = ocean_mask.sel(lon=slice(dincae_coords[0], dincae_coords[1]), lat=slice(dincae_coords[3], dincae_coords[2]))
# ditalymask = ditalymask.astype(bool)
# plt.imshow(ditalymask)
# plt.show()
# print('ditalymask shape:', ditalymask.shape)

#Alternative: mask from the dincae dataset itself
ditalymask = msr_mask = modis_sst_revlat['mask'].values
ditalymask = ditalymask.astype(bool)
ditalymask = np.flip(ditalymask, axis=0)    #Vertical flip
print('ditalymask shape:', ditalymask.shape)
plt.imshow(ditalymask)

In [ ]:
#Dataset creation

# Get a list of all .nc files in the directories and combine all the file lists into one list
all_files = glob.glob('dati/mist/dincae/terra/*.nc')

# Datasets and dates lists (and quality?)
dataset_t = []; date_t = []
qual_t = []

for file in all_files:
    ds = xr.open_dataset(file)

    data = ds['sst4'].values
    data = np.where(ditalymask, data, np.nan)   # Apply the ditalymask

    qual = ds['qual_sst4'].values
    qual = np.where(ditalymask, qual, np.nan)   # Apply the ditalymask
    
    date = pd.to_datetime(ds.attrs['product_name'].split('.')[1]).date()
    date = np.array(date, dtype='datetime64[D]')
    date = date.astype(int)
    
    dataset_t.append(data)
    date_t.append(date)
    qual_t.append(qual)

In [ ]:
# Convert the lists to a numpy array

dataset_t = np.array(dataset_t)
date_t = np.array(date_t)
qual_t = np.array(qual_t)

print(dataset_t.shape)
print(date_t.shape)
print(qual_t.shape)

In [ ]:
print(date_t)

In [ ]:
print("msr_sst_t.shape:", msr_sst_t.shape)
print("msrc_sst_t.shape:", msrc_sst_t.shape)

aaaaaaaaaaa

In [ ]:
msr_sst_t_array = np.load('dati/mist/dincae/msr_sst_t_array.npy')
msrc_sst_t_array = np.load('dati/mist/dincae/msrc_sst_t_array.npy')

print("msr_sst_t_array.shape:", msr_sst_t_array.shape)
print("msrc_sst_t_array.shape:", msrc_sst_t_array.shape)

In [ ]:
#plot msr_sst_t and msrc_sst_t
plt.figure(figsize=(10, 8))
plt.subplot(1, 2, 1)
plt.imshow(msr_sst_t_array[22], cmap='viridis')
plt.title('msr_sst_t')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(msrc_sst_t_array[22], cmap='viridis')
plt.title('msrc_sst_t')
plt.colorbar()
plt.show()


In [ ]:
# cut a sample from msr_sst_t_array and msrc_sst_t_array into a 128x128 image
# the cut should be from the top for the y length, and offset 10 pixels from the right for the x length

cut_msr_sst_t = msr_sst_t_array[:, :128, 30:158]
cut_msrc_sst_t = msrc_sst_t_array[:, :128, 30:158]

print(cut_msr_sst_t.shape)
print(cut_msrc_sst_t.shape)

In [ ]:
#plot cut_msr_sst_t and cut_msrc_sst_t

plt.figure(figsize=(10, 8))
plt.subplot(1, 2, 1)
plt.imshow(cut_msr_sst_t[22], cmap='viridis')
plt.title('cut_msr_sst_t')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(cut_msrc_sst_t[22], cmap='viridis')
plt.title('cut_msrc_sst_t')
plt.colorbar()
plt.show()